In [1]:
# pip install webdriver-manager

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import time
import math

import warnings
warnings.filterwarnings('ignore')

In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

In [4]:
# 크롬 드라이버 실행
driver = webdriver.Chrome(ChromeDriverManager().install())

In [5]:
# 카카오 맵으로 이동
url = "https://map.kakao.com/"
driver.get(url)

In [6]:
searchloc = '전주 한옥숙소'

search_area = driver.find_element(By.XPATH, r'//*[@id="search.keyword.query"]') # 카카오맵 검색창
search_area.send_keys(searchloc)  # 검색어 전달
driver.find_element(By.XPATH, r'//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # 돋보기

time.sleep(2)

driver.find_element(By.XPATH, r'//*[@id="info.main.options"]/li[2]/a').send_keys(Keys.ENTER)  # 장소 탭

In [7]:
room_list = []  # 숙소 정보 저장

In [8]:
def roomNamePrint():
    time.sleep(0.2)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    room_lists = soup.select('.placelist > .PlaceItem') # 장소 정보를 모두 가져옴
    for i, room in enumerate(room_lists):
        temp = []
        
        name = room.select('.head_item > .tit_name > .link_name')[0].text
        score = room.select('.rating > .score > em')[0].text
        addr = room.select('.addr > p')[0].text
        
        # 상세정보 탭으로 이동
        driver.find_element(By.XPATH, r'//*[@id="info.search.place.list"]/li['+str(i+1)+']/div[5]/div[4]/a[1]').send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(2)
        rev = extract_review()  # 리뷰 추출 함수 실행
        
        
        # 하나의 리스트로 만들어 room_list에 추가
        temp.append(name)
        temp.append(score)
        temp.append(addr[3:])
        temp.append(rev)
        
        room_list.append(temp)

In [9]:
def extract_review():
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 후기 목록 찾기
    review_lists = soup.select('.list_evaluation > li')
    
    count = 0
    rev = []
    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for review in review_lists:
            comment = review.select('.txt_comment > span')[0].text  # 리뷰
            if len(comment) != 0:
                rev.append(comment)
    # 없으면 빈칸 추가
    else:
        rev.append(' ')
        
    
    # 다시 검색 탭으로 전환
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(2)
    
    return rev

In [10]:
page = 1  # 현재 페이지
page2 = 1  # 5개 중 몇번째인지(버튼이 5개씩있어서 6번째가 되면 다시 1로 바꿔줘야함)

# 1 ~ 8페이지
for i in range(1, 9):
    try:
        page2 += 1
        print(page, 'page')
        
        # 페이지 버튼 번호(1에서 5 사이 값)
        if i > 5:
            xpath = '/html/body/div[5]/div[2]/div[1]/div[7]/div[6]/div/a['+str(i-5)+']'
        else:
            xpath = '/html/body/div[5]/div[2]/div[1]/div[7]/div[6]/div/a['+str(i)+']'
        
        driver.find_element(By.XPATH, xpath).send_keys(Keys.ENTER)  # 페이지 선택
        roomNamePrint()  # 숙소 정보 크롤링
        
        
        # page2가 5를 넘어가면 다시 1로 바꿔주고 다음 버튼 클릭
        if page2 > 5:
            page2 = 1
            driver.find_element(By.XPATH, r'//* [@id="info.search.page.next"]').send_keys(Keys.ENTER)
        
        page += 1
    
    except:
        break
        
print('크롤링 완료')

1 page
2 page
3 page
4 page
5 page
6 page
7 page
8 page
크롤링 완료


In [11]:
room_list

[['나비잠한옥호텔',
  '4.5',
  '전주시 완산구 팔달로 150-17',
  ['침구상태가 최상은 아니었어요. 또 휴지가 욕실에만 있고 객실에는 없어서 불편한 점이 있었으나 방마다 욕실이 있고, 친절하셔서 잘 보냈어요. 위치도 좋고요.',
   '친절하시고 침대가 생각보다 아주 편합니다. 방도 굉장히 깨끗하구요']],
 ['한옥호텔궁',
  '3.6',
  '전주시 완산구 풍남문4길 25-19',
  ['평일이라 룸 업그레이드도 받고 정말 조용하게 편하게 쉬었다 왔습니다. 룸 컨디션도 최상으로 유지되었구요. 담에도 또 방문하겠습니다. 다만 사진을 올리지 못해서 ㅠ']],
 ['이가한옥',
  '3.9',
  '전주시 완산구 간납로 8',
  ['숙소가 한옥이라 분위기 있음.작지만 뒷편에 주차장도 있음.샤워기 수압 쎄고 따뜻한 물, 난방 잘 됨~but 방이 진짜 좁고.. 정말 방음이 단 1도 안된다는 거!진짜 바로 옆에 있는 듯 모든 소리가 다 들림 ㅋ욕실 사용도 살짝 불편했음.하지만 사장님 정말 교양있고 친절하심~여기 숙박하면 쎄미한복에서 대여 할인해줌',
   '숙박 얼만지 전화해서 물어보니까 검색하면 나오는거 왜물어보냐고 불쾌해하시는데 목소리만 친절한척하는 말이나 행동은 너무불친절하시네요방음도안되고 방도좁고 이럴거면 모텔을가는걸 추천!']],
 ['정가한옥',
  '3.8',
  '전주시 완산구 기린대로 78-3',
  ['어떤 방이 30분 넘게 시끄러워서 퇴실 시간인 줄 알았는데 새벽 0시 50분..시간 가늠하려 방문 열었는데 담배물고 방에서 나오는 센스하며저 정도 소음이면 문을 열고 떠들었거나문을 닫고 학교 쉬는 시간처럼 떠든건데미친것 같아요. 개념이 1도 없네요. 만취가 아니라면 이해가 안가요.가족이 안 말렸으면 정말 싹 쓸어버리고 싶을 정도네요문제 해결을 위한 방법을 고안하시면 4.5점 이상의 숙소가 될 것이라 생각합니다',
   '우연한 기회에 연이 닿은 정가한옥과 더정가스테이예요^^저희는 신축건물인 더정가스테이에 묵었지만 정가한옥도 좋을

In [12]:
room_list[0]

['나비잠한옥호텔',
 '4.5',
 '전주시 완산구 팔달로 150-17',
 ['침구상태가 최상은 아니었어요. 또 휴지가 욕실에만 있고 객실에는 없어서 불편한 점이 있었으나 방마다 욕실이 있고, 친절하셔서 잘 보냈어요. 위치도 좋고요.',
  '친절하시고 침대가 생각보다 아주 편합니다. 방도 굉장히 깨끗하구요']]

In [13]:
room_list[0][3] # 리뷰

['침구상태가 최상은 아니었어요. 또 휴지가 욕실에만 있고 객실에는 없어서 불편한 점이 있었으나 방마다 욕실이 있고, 친절하셔서 잘 보냈어요. 위치도 좋고요.',
 '친절하시고 침대가 생각보다 아주 편합니다. 방도 굉장히 깨끗하구요']

In [14]:
name = []
score = []
address = []
review = []
for r in room_list:
    name.append(r[0])
    score.append(r[1])
    address.append(r[2])
    review.append(r[3])

In [15]:
hanok = pd.DataFrame({'name':name, 'score':score, 'address':address, 'review':review})
hanok

,name,score,address,review
0,나비잠한옥호텔,4.5,전주시 완산구 팔달로 150-17,[침구상태가 최상은 아니었어요. 또 휴지가 욕실에만 있고 객실에는 없어서 불편한 점...
1,한옥호텔궁,3.6,전주시 완산구 풍남문4길 25-19,[평일이라 룸 업그레이드도 받고 정말 조용하게 편하게 쉬었다 왔습니다. 룸 컨디션도...
2,이가한옥,3.9,전주시 완산구 간납로 8,[숙소가 한옥이라 분위기 있음.작지만 뒷편에 주차장도 있음.샤워기 수압 쎄고 따뜻한...
3,정가한옥,3.8,전주시 완산구 기린대로 78-3,[어떤 방이 30분 넘게 시끄러워서 퇴실 시간인 줄 알았는데 새벽 0시 50분..시...
4,라온한옥꿀잠,4.3,전주시 완산구 한지길 99-6,[]
...,...,...,...,...
104,해오름펜션,0.0,전주시 완산구 전주천동로 52,[ ]
105,행복헌,0.0,전주시 완산구 은행로 83-20,[ ]
106,두울나비랑,0.0,전주시 완산구 향교길 97-16,[ ]
107,행진게스트하우스,5.0,전주시 완산구 향교길 155-20,"[내집처럼 편안하고 바로 근처에 향교, 전주천, 생태원 등 산책하기 좋은 곳이 많아요.]"


In [16]:
hanok.to_csv('hanok.csv', index=False, encoding="utf-8-sig")